In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Supply Chain Data/DataCoSupplyChainDataset.csv')

In [3]:
# y value delivery status, categorically able to predict if a delivery is
# late, advanced/ a head of schedule, on time, or was canceled. 
df['Delivery Status'].value_counts()

Late delivery        98977
Advance shipping     41592
Shipping on time     32196
Shipping canceled     7754
Name: Delivery Status, dtype: int64

In [4]:
df.drop(columns=[
    'Category Name',
    'Customer City',
    'Customer Country',
    'Customer Email',
    'Customer Fname',
    'Customer Id',
    'Customer Lname',
    'Customer Password',
    'Customer Street',
    'Order Item Cardprod Id',
    'Product Card Id',
    'Product Category Id',
    'Product Description',
    'Product Status',
    'Product Image',
    'Product Name',
    'Order Zipcode'
], axis=1, inplace=True)

In [5]:
for c in df.columns:
    print(c)
    print(df[c].isna().sum())

Type
0
Days for shipping (real)
0
Days for shipment (scheduled)
0
Benefit per order
0
Sales per customer
0
Delivery Status
0
Late_delivery_risk
0
Category Id
0
Customer Segment
0
Customer State
0
Customer Zipcode
3
Department Id
0
Department Name
0
Latitude
0
Longitude
0
Market
0
Order City
0
Order Country
0
Order Customer Id
0
order date (DateOrders)
0
Order Id
0
Order Item Discount
0
Order Item Discount Rate
0
Order Item Id
0
Order Item Product Price
0
Order Item Profit Ratio
0
Order Item Quantity
0
Sales
0
Order Item Total
0
Order Profit Per Order
0
Order Region
0
Order State
0
Order Status
0
Product Price
0
shipping date (DateOrders)
0
Shipping Mode
0


In [6]:
df[df['Customer Zipcode'].isna() == True]

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Customer Segment,Customer State,...,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Price,shipping date (DateOrders),Shipping Mode
35704,DEBIT,2,1,66.379997,189.660004,Late delivery,1,65,Consumer,95758,...,1,252.880005,189.660004,66.379997,Western Europe,Auvernia-R�dano-Alpes,COMPLETE,252.880005,11/4/2017 18:31,First Class
46440,PAYMENT,4,4,10.910000,38.959999,Shipping on time,0,75,Corporate,95758,...,1,39.750000,38.959999,10.910000,Southeast Asia,Capital Nacional,PENDING_PAYMENT,39.750000,12/14/2017 15:18,Standard Class
82511,DEBIT,4,4,59.990002,299.959992,Shipping on time,0,63,Consumer,91732,...,1,357.100006,299.959992,59.990002,Northern Europe,Inglaterra,COMPLETE,357.100006,10/30/2017 0:29,Standard Class


In [7]:
#clean customer zipcode
df.drop(index=[35704, 46440, 82511], axis=0, inplace=True)

##### Converting Date to Year-Month

In [8]:
#order date
date_od = pd.to_datetime(pd.Series(df['order date (DateOrders)'], )).dt.strftime('%Y-%m')

In [9]:
df = pd.concat([df, date_od], axis=1)
df['order date y-m'] = df['order date (DateOrders)'].iloc[:, 1]
df.drop(columns=['order date (DateOrders)'], axis=1, inplace=True)

In [10]:
#shipping date
date_sd = pd.to_datetime(pd.Series(df['shipping date (DateOrders)'])).dt.strftime('%Y-%m')

In [11]:
df = pd.concat([df, date_sd], axis=1)
df['shipping date y-m'] = df['shipping date (DateOrders)'].iloc[:, 1]
df.drop(columns=['shipping date (DateOrders)'], axis=1, inplace=True)

In [12]:
#dropping shipping cancelled rows. 
df = df[df['Delivery Status'] != 'Shipping canceled' ]

In [13]:
#reset index
df.reset_index(drop=True, inplace=True)

In [14]:
df.dropna(axis=0, inplace=True)

In [15]:
df.sort_values(by=['order date y-m', 'shipping date y-m'], inplace=True)

In [16]:
df = df.drop(columns=[
    'Days for shipping (real)',
    'Benefit per order',
    'Category Id',
    'Customer State',
    'Order State',
    'Order Item Total',
    'Order Customer Id',
    'Order Id',
    'Order City',
    'Order Country',
    'Order Status',
    'Product Price',
], axis=1)

In [17]:
df.to_csv('df.csv')

In [18]:
#setting advance shipping status to on time to see if the model is 
#predicitng simply if we care the package is on time or not.

df_ontime_vs_late = df.replace(to_replace='Advance shipping', value='Shipping on time')
df_ontime_vs_late

,Type,Days for shipment (scheduled),Sales per customer,Delivery Status,Late_delivery_risk,Customer Segment,Customer Zipcode,Department Id,Department Name,Latitude,...,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Profit Per Order,Order Region,Shipping Mode,order date y-m,shipping date y-m
101,TRANSFER,4,167.960007,Shipping on time,0,Consumer,725.0,5,Golf,18.253332,...,2420,39.990002,0.50,5,199.949997,83.980003,Caribbean,Standard Class,2015-01,2015-01
131,TRANSFER,4,283.450012,Late delivery,1,Consumer,725.0,4,Apparel,18.292149,...,989,59.990002,-0.55,5,299.950012,-155.899994,Central America,Standard Class,2015-01,2015-01
314,TRANSFER,4,71.980003,Shipping on time,0,Corporate,94541.0,5,Golf,37.675510,...,922,39.990002,0.33,2,79.980003,23.389999,South America,Standard Class,2015-01,2015-01
637,TRANSFER,4,169.960007,Shipping on time,0,Consumer,725.0,5,Golf,18.202093,...,3424,39.990002,0.08,5,199.949997,13.260000,South America,Standard Class,2015-01,2015-01
645,TRANSFER,4,230.350006,Late delivery,1,Consumer,725.0,6,Outdoors,18.215712,...,2749,47.990002,-0.06,5,239.949997,-14.280000,Caribbean,Standard Class,2015-01,2015-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166386,CASH,4,209.350006,Late delivery,1,Consumer,725.0,4,Apparel,18.289610,...,180475,215.820007,0.26,1,215.820007,54.430000,Eastern Asia,Standard Class,2018-01,2018-02
166433,CASH,4,183.449997,Late delivery,1,Consumer,725.0,4,Apparel,18.239115,...,180160,215.820007,0.35,1,215.820007,64.209999,Southeast Asia,Standard Class,2018-01,2018-02
166455,CASH,4,176.970001,Late delivery,1,Consumer,725.0,4,Apparel,18.209925,...,180409,215.820007,0.38,1,215.820007,66.360001,South Asia,Standard Class,2018-01,2018-02
168052,DEBIT,1,183.449997,Late delivery,1,Consumer,725.0,4,Apparel,18.229307,...,180394,215.820007,0.28,1,215.820007,51.369999,South Asia,First Class,2018-01,2018-02


In [19]:
#now only late delivery and shipping on time are considered in this dataframe
df_ontime_vs_late['Delivery Status'].value_counts()

Late delivery       98976
Shipping on time    73786
Name: Delivery Status, dtype: int64

In [20]:
df_ontime_vs_late.to_csv('df_ontime_vs_late.csv')

In [21]:
len(df_ontime_vs_late)

172762